# **VIDEO GENERATION**

In [13]:
import json
import os
import requests
import replicate
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

if os.environ.get('REPLICATE_API_TOKEN'):
    print("✅ Replicate API key loaded successfully")
else:
    raise ValueError("ERROR: Missing REPLICATE_API_TOKEN in .env")

with open("current_strategy.json", "r") as f:
    strategy = json.load(f)



✅ Replicate API key loaded successfully


In [14]:

def generate_video(prompt):
    print(f"🎬 Starting video generation...")
    print(f"📝 Prompt: {prompt[:100]}...")
    prediction = replicate.predictions.create(
        model="minimax/video-01",
        input={
            "prompt": prompt,
            "prompt_optimizer": True
        }
    )
    print(f"⏳ Waiting for video... (ID: {prediction.id})")
    prediction.wait()
    if prediction.status == "succeeded":
        print(f"✅ Video generated!")
        return prediction.output
    else:
        raise Exception(f"Generation failed: {prediction.error}")

def download_video(video_url, output_dir="output_video"):
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    video_path = f"{output_dir}/{timestamp}_video.mp4"
    
    print(f"📥 Downloading video...")
    response = requests.get(video_url, stream=True)
    response.raise_for_status()
    
    with open(video_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    
    print(f"💾 Video saved to: {video_path}")
    return video_path

In [15]:
video_prompt = strategy["video_prompt"]

video_output = generate_video(video_prompt)
video_url = str(video_output)
video_path = download_video(video_url)
result = {
    "video_path": video_path,
    "video_url": video_url,
    "strategy": strategy,
    "generated_at": datetime.now().isoformat()
}

with open("output_video/video_metadata.json", "w") as f:
    json.dump(result, f, indent=2)

print("✅ Complete! Video and metadata saved to output_video/")

🎬 Starting video generation...
📝 Prompt: The camera executes a slow, deliberate dolly-in, starting from a medium shot of a sleek, modern smar...
⏳ Waiting for video... (ID: p6az61w6shrmy0cw5s7srjxdpm)
✅ Video generated!
📥 Downloading video...
💾 Video saved to: output_video/2026-02-05_20-30-40_video.mp4
✅ Complete! Video and metadata saved to output_video/
